<a href="https://colab.research.google.com/github/sidharrth2002/stanford-cars/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing and Augmenting Stanford Cars

Bounding boxes are provided by Stanford, we use them to crop and augment images before saving them in folders, that can be loaded in by Keras ImageDataGenerator.

In [79]:
import tarfile
import os
import csv
import matplotlib.pyplot as plt
from scipy.io import loadmat
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
import PIL
from tqdm import tqdm
%matplotlib inline

In [4]:
!wget --no-check-certificate http://ai.stanford.edu/~jkrause/car196/cars_train.tgz 

train_tar = tarfile.open('cars_train.tgz')
train_tar.extractall('./train') # specify which folder to extract to
train_tar.close()

--2022-04-30 10:44:28--  http://ai.stanford.edu/~jkrause/car196/cars_train.tgz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 979269282 (934M) [application/x-gzip]
Saving to: ‘cars_train.tgz.1’

cars_train.tgz.1    100%[===================>] 933.90M  38.5MB/s    in 28s     

2022-04-30 10:44:56 (33.6 MB/s) - ‘cars_train.tgz.1’ saved [979269282/979269282]



In [6]:
#Download full testing data (~933MB)
!wget --no-check-certificate http://ai.stanford.edu/~jkrause/car196/cars_test.tgz #downloaded directory
#extract testing data to tmp/test/
test_tar = tarfile.open('cars_test.tgz')
test_tar.extractall('./test') # specify which folder to extract to
test_tar.close()

--2022-04-30 10:47:21--  http://ai.stanford.edu/~jkrause/car196/cars_test.tgz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 977350468 (932M) [application/x-gzip]
Saving to: ‘cars_test.tgz’

cars_test.tgz       100%[===================>] 932.07M  38.1MB/s    in 28s     

2022-04-30 10:47:49 (33.3 MB/s) - ‘cars_test.tgz’ saved [977350468/977350468]



In [9]:
#Download labels (~0.3MB)
!wget --no-check-certificate https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz -O car_devkit.tgz
Path("/tmp/annos").mkdir(parents=True, exist_ok=True)
!wget --no-check-certificate \
    https://people.ucsc.edu/~jwang402/carslabel/anno_test.csv \
    -O anno_test.csv
#extract labels to /tmp/labels/
label_tar = tarfile.open('car_devkit.tgz')
label_tar.extractall('labels') # specify which folder to extract to
label_tar.close()

--2022-04-30 10:49:59--  https://ai.stanford.edu/~jkrause/cars/car_devkit.tgz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 330960 (323K) [application/x-gzip]
Saving to: ‘car_devkit.tgz’

car_devkit.tgz      100%[===================>] 323.20K   505KB/s    in 0.6s    

2022-04-30 10:50:00 (505 KB/s) - ‘car_devkit.tgz’ saved [330960/330960]

--2022-04-30 10:50:00--  https://people.ucsc.edu/~jwang402/carslabel/anno_test.csv
Resolving people.ucsc.edu (people.ucsc.edu)... 128.114.119.88
Connecting to people.ucsc.edu (people.ucsc.edu)|128.114.119.88|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225299 (220K) [text/csv]
Saving to: ‘anno_test.csv’

anno_test.csv       100%[===================>] 220.02K   785KB/s    in 0.3s    

2022-04-30 10:50:01 (785 KB/s) - ‘anno_test.csv’ saved [225299/225299]



In [11]:
meta = loadmat('/content/labels/devkit/cars_meta.mat')
meta.keys()

dict_keys(['__header__', '__version__', '__globals__', 'class_names'])

In [42]:
classes = [[row.flat[0] for row in line] for line in meta['class_names'][0]]
classDict = {}
classCount = 1
for c in classes:
  classDict.update({classCount:c[0]})
  classCount += 1
counter = 0
for num, model in classDict.items():
  print(f'Car Model {num}: {model}')
  counter += 1
print('...')
print(f'There are a total of {len(classDict.items())} cars')

Car Model 1: AM General Hummer SUV 2000
Car Model 2: Acura RL Sedan 2012
Car Model 3: Acura TL Sedan 2012
Car Model 4: Acura TL Type-S 2008
Car Model 5: Acura TSX Sedan 2012
Car Model 6: Acura Integra Type R 2001
Car Model 7: Acura ZDX Hatchback 2012
Car Model 8: Aston Martin V8 Vantage Convertible 2012
Car Model 9: Aston Martin V8 Vantage Coupe 2012
Car Model 10: Aston Martin Virage Convertible 2012
Car Model 11: Aston Martin Virage Coupe 2012
Car Model 12: Audi RS 4 Convertible 2008
Car Model 13: Audi A5 Coupe 2012
Car Model 14: Audi TTS Coupe 2012
Car Model 15: Audi R8 Coupe 2012
Car Model 16: Audi V8 Sedan 1994
Car Model 17: Audi 100 Sedan 1994
Car Model 18: Audi 100 Wagon 1994
Car Model 19: Audi TT Hatchback 2011
Car Model 20: Audi S6 Sedan 2011
Car Model 21: Audi S5 Convertible 2012
Car Model 22: Audi S5 Coupe 2012
Car Model 23: Audi S4 Sedan 2012
Car Model 24: Audi S4 Sedan 2007
Car Model 25: Audi TT RS Coupe 2012
Car Model 26: BMW ActiveHybrid 5 Sedan 2012
Car Model 27: BMW 1 S

In [44]:
labels = loadmat('./labels/devkit/cars_train_annos.mat')

trainLabels = [[row.flat[0] for row in line] for line in labels['annotations'][0]]
columns = ['x1', 'y1', 'x2', 'y2', 'class', 'file_name']
bounding_labels = pd.DataFrame(trainLabels, columns=columns)
bounding_labels.head(10)

,x1,y1,x2,y2,class,file_name
0,39,116,569,375,14,00001.jpg
1,36,116,868,587,3,00002.jpg
2,85,109,601,381,91,00003.jpg
3,621,393,1484,1096,-122,00004.jpg
4,14,36,133,99,106,00005.jpg
5,259,289,515,416,123,00006.jpg
6,88,80,541,397,89,00007.jpg
7,73,79,591,410,96,00008.jpg
8,20,126,1269,771,-89,00009.jpg
9,21,110,623,367,58,00010.jpg


In [70]:
lf = loadmat('/content/labels/devkit/cars_test_annos.mat')
testLabels = [[row.flat[0] for row in line] for line in lf['annotations'][0]]

#Test labels class and fname
with open('anno_test.csv') as csv_file:
    read = csv.reader(csv_file, delimiter=',')
    testDict = {}
    classCount = 1
    for row in read:
        testDict.update({row[0]:row[5]})
        classCount = classCount + 1
    print(len(testDict), 'processed')

columns = ['x1', 'y1', 'x2', 'y2', 'file_name']
testing = pd.DataFrame(testLabels, columns=columns)
testing.head(10)

8041 processed


,x1,y1,x2,y2,file_name
0,30,52,246,147,00001.jpg
1,100,19,576,203,00002.jpg
2,51,105,968,659,00003.jpg
3,67,84,581,407,00004.jpg
4,140,151,593,339,00005.jpg
5,20,77,420,301,00006.jpg
6,249,166,2324,1459,00007.jpg
7,119,215,1153,719,00008.jpg
8,1,7,275,183,00009.jpg
9,28,55,241,177,00010.jpg


In [51]:
bounding_labels['class'] = bounding_labels['class'].apply(lambda x: x + 256 if x < 0 else x)

In [53]:
bounding_labels['class'].max()

196

In [55]:
classDict.keys()

dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196])

In [56]:
print(f"There are {len(trainLabels)} images in the training set.")

There are 8144 images in the training set.


In [57]:
training, validation = train_test_split(bounding_labels, stratify=bounding_labels['class'], test_size=0.2)
print(f'There are {len(training)} in training set.')
print(f'There are {len(validation)} in validation set.')

There are 6515 in training set.
There are 1629 in validation set.


In [63]:
from re import L
'''
I will save to our shared drive folder.
This cell will crop the images based on their bounding boxes, and append 
stratified 20% of all images to a validation set that is used for hyperparameter tuning.
'''
train_save_path = '/content/drive/Shareddrives/ML/preprocessed/train'
val_save_path = '/content/drive/Shareddrives/ML/preprocessed/val'
Path(train_save_path).mkdir(parents=True, exist_ok=True)
Path(val_save_path).mkdir(parents=True, exist_ok=True)

progress = tqdm(total=len(training))
counter = 0
print('Processing and cropping training images')
for index, row in training.iterrows():
  x1 = row['x1']
  y1 = row['y1']
  x2 = row['x2']
  y2 = row['y2']
  class_name = row['class']
  file_name = row['file_name']
  location = f'./train/cars_train/{file_name}'
  class_label = classDict.get(class_name) + '/'
  saveDir = f'{train_save_path}/{class_label}'
  image = PIL.Image.open(location)
  region = image.crop((x1, y1, x2, y2))
  Path(saveDir).mkdir(parents=True, exist_ok=True)
  saveAddress = saveDir + file_name
  region.save(saveAddress)
  counter += 1
  progress.update(1)

progress.close()
print('Finished processing and cropping training images')
print('All images have been saved to ', train_save_path)

  5%|▌         | 330/6515 [00:19<06:06, 16.85it/s]


Processing and cropping training images


100%|██████████| 6515/6515 [03:21<00:00, 32.37it/s]

Finished processing and cropping training images
All images have been saved to  /content/drive/Shareddrives/ML/preprocessed/train


In [64]:
progress = tqdm(total=len(validation))
counter = 0
print('Processing and cropping validation images')
for index, row in validation.iterrows():
  x1 = row['x1']
  y1 = row['y1']
  x2 = row['x2']
  y2 = row['y2']
  class_name = row['class']
  file_name = row['file_name']
  location = f'./train/cars_train/{file_name}'
  class_label = classDict.get(class_name) + '/'
  saveDir = f'{val_save_path}/{class_label}'
  image = PIL.Image.open(location)
  region = image.crop((x1, y1, x2, y2))
  Path(saveDir).mkdir(parents=True, exist_ok=True)
  saveAddress = saveDir + file_name
  region.save(saveAddress)
  counter += 1
  progress.update(1)

progress.close()
print('Finished processing and cropping validation images')
print('All images have been saved to ', val_save_path)

  0%|          | 0/1629 [00:00<?, ?it/s]

Processing and cropping validation images


100%|██████████| 1629/1629 [00:47<00:00, 34.43it/s]

Finished processing and cropping validation images
All images have been saved to  /content/drive/Shareddrives/ML/preprocessed/val


In [80]:
"""
Preprocess Testing Images
Gets all training images and crops by bounding box, saves to a new directory
"""
test_save_path = '/content/drive/Shareddrives/ML/preprocessed/test_4'
test_save_path_uncropped = '/content/drive/Shareddrives/ML/preprocessed/test_uncropped_4'
Path(test_save_path).mkdir(parents=True, exist_ok=True)
print(len(testLabels), "images to process.")

count = 1
progress = tqdm(total=len(testLabels))
for l in testLabels:
    x1 = l[0]
    y1 = l[1]
    x2 = l[2]
    y2 = l[3]
    filename = l[4]
    c_num = testDict.get(filename)
    classAppend = classDict.get(int(c_num)) + '/'
    address = "./test/cars_test/" + filename
    saveDir = test_save_path + '/' + classAppend
    saveDir_uncropped = test_save_path_uncropped + '/' + classAppend
    im = PIL.Image.open(address)
    region = im.crop((x1, y1, x2, y2))
    Path(saveDir).mkdir(parents=True, exist_ok=True)
    Path(saveDir_uncropped).mkdir(parents=True, exist_ok=True)
    saveAddress = saveDir + filename
    saveAddress2 = saveDir_uncropped + filename
    im.save(saveAddress2)
    region.save(saveAddress)    
    count = count + 1
    progress.update(1)
progress.close()
print('Finished processing and cropping testing images')
print('All processed images have been saved to ', test_save_path)
print('All raw images have been saved to ', test_save_path_uncropped)

8041 images to process.


100%|██████████| 8041/8041 [07:25<00:00, 18.05it/s]

Finished processing and cropping testing images
All processed images have been saved to  /content/drive/Shareddrives/ML/preprocessed/test_4
All raw images have been saved to  /content/drive/Shareddrives/ML/preprocessed/test_uncropped_4
